In [1]:
from skimage import io as skio
from scipy.optimize import least_squares
import math
from tqdm import tqdm
import numpy as np

from matplotlib import pyplot as plt
import pickle
from skimage.transform import resize

from scipy import interpolate

from scipy.interpolate import interp2d

from skimage.draw import polygon





In [9]:
from glob import glob

In [57]:
BIG_K = 20
DISTRIBUTION = np.array([0.17, 0.15, 0.13, 0.11, 0.1, 0.1, 0.07, 0.07, 0.05, 0.05])
DISTRIBUTION = np.hstack([DISTRIBUTION*0.67, DISTRIBUTION * 0.33])
DISTRIBUTION = (list(DISTRIBUTION))
DISTRIBUTION.sort(reverse=True)
sum(DISTRIBUTION)

1.0000000000000002

In [58]:
DISTRIBUTION

[0.11390000000000002,
 0.1005,
 0.08710000000000001,
 0.0737,
 0.067,
 0.067,
 0.056100000000000004,
 0.0495,
 0.046900000000000004,
 0.046900000000000004,
 0.0429,
 0.0363,
 0.0335,
 0.0335,
 0.033,
 0.033,
 0.023100000000000002,
 0.023100000000000002,
 0.0165,
 0.0165]

In [3]:
frame_sequence = []

curr = (10, 10)


In [4]:
import os

In [5]:
import scipy.io

In [6]:
top_breeds = ['005.alaskan_malamute','015.basset_hound','029.border_collie','039.bull_terrier','057.dalmatian','041.bullmastiff','087.irish_terrier','056.dachshund','016.beagle','012.australian_shepherd']

with open('sfm_warp_data/camera_mts.pickle', 'rb') as handle:
    camera_mts = pickle.load(handle)
with open('sfm_warp_data/3d_points.pickle', 'rb') as handle:
    points_meanaligned_3d = pickle.load(handle)

In [7]:
def add_corners_2(ptset):
    corners = np.array([[0, 0, 223, 223], [0, 223, 0, 223]])
    return np.hstack([ptset, corners])

def add_corners_3(ptset):
    corners = np.array([[0, 0, 223, 223], [0, 223, 0, 223], [0, 0, 0, 0]])
    return np.hstack([ptset, corners])   

In [10]:
filenames = {}
for BREED in top_breeds:

    breed_mats = glob(f'CU_dogs/training_133/{BREED}/*v5.mat')

    breed_mats.sort()

    filenames[BREED] = breed_mats



### code for morphing


In [11]:
simplices=[
        [12, 16, 14],
    [12, 11, 16],
    [11,  9, 16],
    [16,  9, 15],
    [11,  1,  9],
    [ 1,  5,  9],
    [ 9,  5, 15],
    [ 1, 11,  8],
    [ 5,  1,  8],
   
        [ 4,  5,  8],
    [ 5,  4, 15],
    [ 7,  4,  8],
     [11,  2,  8],
    [ 2,  7,  8],
    [ 7,  2, 10],
    [10, 12, 14],


        [ 6, 14, 13],
    [15,  4, 13],
    [ 3,  6, 13],
    [ 4,  3, 13],
    [ 6, 10, 14],
    [ 7,  3,  4],
    [ 0, 10,  6],
    [ 0,  7, 10],
    [ 3,  0,  6],
    [ 7,  0,  3],
    
            [ 2, 11, 12],
    [10,  2, 12],

]

In [12]:
"""
use inverse warp to do one triangle, in one direction
params:
1. simplex - the 3 indices connected by the triangle
2. pts_1 - original point coordinates
3. pts_2 - the new points

returns:
a matrix : ) 
"""
def calculate_one_triangle(simplex, pts_1, pts_2, src_img, dest_img):
    
    # get the triangle corners
    orig_points = pts_1[simplex]
    new_points = pts_2[simplex]
    
    # get the polygon index for source and dest
    old_poly = polygon(orig_points[:,1], orig_points[:,0])
    newpoly = polygon(new_points[:,1], new_points[:,0])
    
    
    
    # get the pixel values for the old image
    r_old = src_img[:,:,0][old_poly]
    g_old = src_img[:,:,1][old_poly]
    b_old = src_img[:,:,2][old_poly]
    
    # get the inverse warp matrix
    tmx = computeAffine(simplex, pts_1, pts_2)
    tmx_i = np.linalg.inv(tmx)
    
    # do the inverse warp
    dest_pixels_matrix = np.vstack((newpoly[1], newpoly[0], np.ones(len(newpoly[0]))))
    inverse_warp = (tmx_i @ dest_pixels_matrix)[:2]
    # interpolate
    try:
        r_new = interpolate.griddata((old_poly), r_old, (inverse_warp[1], inverse_warp[0]), method='nearest')
    except:
        r_new = np.zeros(len(newpoly[0]))
    try:
        g_new = interpolate.griddata((old_poly), g_old, (inverse_warp[1], inverse_warp[0]), method='nearest')
    except:
        g_new = np.zeros(len(newpoly[0]))
    try:
        b_new = interpolate.griddata((old_poly), b_old, (inverse_warp[1], inverse_warp[0]), method='nearest')
    except:
        b_new = np.zeros(len(newpoly[0]))
    
    triangle = np.vstack((r_new, g_new,b_new)).T
    triangle = np.clip(triangle, 0, 1)
    
    #apply pixels to dest image
    dest_img[newpoly[0],newpoly[1]] = triangle


def computeAffine(simplex, pts_1, pts_2):
    orig_points = pts_1[simplex]

    new_points = pts_2[simplex]

    b = new_points
    A = np.hstack([orig_points, np.ones(3).reshape((3,1))])

    x = np.linalg.solve(A, b)

    tmx = np.vstack([x.T, [0,0,1]])
    return tmx

"""
this is the same code as the previous section, just with some names changed
"""

def morph(im1, im2, im1_pts, im2_pts, simplices, warp_frac, dissolve_frac):
    
    assert im1.shape == im2.shape
    assert len(im1_pts) == len(im2_pts)

    target_points = (im1_pts * warp_frac) + (im2_pts * (1 - warp_frac))
    
    im1_warped = np.zeros(im1.shape, dtype = float)
    im2_warped = np.zeros(im1.shape, dtype = float)
    
    for idx, simplex in enumerate(simplices):
        calculate_one_triangle(simplex, im1_pts, target_points, im1, im1_warped)
        calculate_one_triangle(simplex, im2_pts, target_points, im2, im2_warped)
        
    combined = im1_warped*dissolve_frac + im2_warped*(1-dissolve_frac)
    return combined

### the path which the animation will take

In [13]:
path = []
rad = 1
curr = [10 - 1, 10 - 1]
path.append(curr)
while True:
    
    for _ in range(rad * 2 - 1):
        curr = [curr[0] + 1, curr[1]]
        path.append(curr)
    for _ in range(rad * 2 - 1):
        curr = [curr[0], curr[1] + 1]
        path.append(curr)
    for _ in range(rad * 2 - 1):
        curr = [curr[0] - 1, curr[1]]
        path.append(curr)
    for _ in range(rad * 2 - 2):
        curr = [curr[0], curr[1] - 1]
        path.append(curr)
    curr = [curr[0] - 1, curr[1] - 2]

    if curr[0] < 0 or curr[1] < 0:
        break
    path.append(curr)
    
    rad += 1



### the rest

In [14]:
def angle_estimation_loss(arr, orig, target):
    
    # arr: alpha, beta, gamma, scale
    
    tmx, T = reconstruct_tmx(arr)
    rotated_2d = (tmx @ orig)[:2]
    
    translated = rotated_2d + T
    errors = (translated - target) ** 2
    errors = np.mean(errors, axis = 0)
    
    return errors
    
def reconstruct_tmx(arr):
    alpha, beta, gamma, scale, tx, ty = arr
    ca, cb, cg = math.cos(alpha), math.cos(beta), math.cos(gamma)
    sa, sb, sg = math.sin(alpha), math.sin(beta), math.sin(gamma)
    rotation = [[ca * cb, (ca*sb*sg) - (sa*cg), (ca*sb*cg) + (sa*sg)],
                [sa*cb, (sa*sb*sg) + (ca*cg), (sa*sb*cg) - (ca*sg)],
                [- sb, cb * sg, cb * cg]]
    
    rmx = np.array(rotation)
    rmx = rmx * scale
    
    T = np.array([[tx],[ty]])
    
    return rmx, T

In [60]:
def get_top_k(available, target, k = BIG_K):
    target = np.array(target)
    errs = available - target.reshape((2, 1))
    errs = np.sum((errs ** 2), axis = 0)
    return np.argsort(errs)[:k]

In [16]:
def grab_pic(breed, index):
    mat = filenames[BREED][index]
    mat = scipy.io.loadmat(mat)
    return np.array(mat['Imask']), np.array(mat['points']) 

REMEMBEr

gamma - rotation around X axis (look up / look down)

beta - rotation around Y axis (look left/ look right)

alpha - rotation around Z axis (tilt da head)

In [17]:
for breed in top_breeds:
    print(f"{breed}:{len(camera_mts[breed])}")

005.alaskan_malamute:57
015.basset_hound:54
029.border_collie:52
039.bull_terrier:50
057.dalmatian:49
041.bullmastiff:48
087.irish_terrier:47
056.dachshund:47
016.beagle:42
012.australian_shepherd:47


In [23]:
BREED = top_breeds[5]

alphas = [d['alpha'] for d in camera_mts[BREED]]
betas = [d['beta'] for d in camera_mts[BREED]]
gammas = [d['gamma'] for d in camera_mts[BREED]]
translates = [d['T'] for d in camera_mts[BREED]]
scales = [d['scale'] for d in camera_mts[BREED]]

betas_and_gammas = np.array([betas, gammas])

In [385]:
os.makedirs(f"frames_tilt/{BREED}", exist_ok=True)

# grid: each row is one gamma. each col is one beta
grid_of_topks = np.empty((20,20), dtype=object)
for col, beta in enumerate(np.linspace(min(betas), max(betas), 20)):
    for row, gamma in enumerate(np.linspace(min(gammas), max(gammas), 20)):
        indices = get_top_k(betas_and_gammas, [beta, gamma])
        grid_of_topks[row, col] = {
            'indices': indices,
            'beta': beta,
            'gamma': gamma
        }

# do morphing

for frame_idx, (do_row, do_col) in tqdm(enumerate(path)):

    d = grid_of_topks[do_row, do_col]

    indices, beta, gamma = d['indices'], d['beta'], d['gamma']

    three_d = add_corners_3(points_meanaligned_3d[BREED])

    rotmx = reconstruct_tmx([0, beta,gamma, 1, 0, 0])[0]
    ctr_pt = rotmx @ np.array([[112],[112],[0]])
    three_d_spun = rotmx@three_d

    new_width = three_d_spun[0, 15] - three_d_spun[0, 13]
    new_height = three_d_spun[1, 14] - three_d_spun[1, 13]

    # adjust the thing

    scale_factor = 224/((new_width + new_height)/2)

    three_d_spun, ctr_pt = three_d_spun * scale_factor, ctr_pt * scale_factor
    three_d_spun = three_d_spun - ctr_pt + np.array([[112],[112],[0]])

    flattened_targ = three_d_spun[:2]
    flattened_targ = np.minimum(223, flattened_targ)
    flattened_targ = np.maximum(0, flattened_targ)

    warpeds = []
    for idx in range(BIG_K):

        img, opoints = grab_pic(BREED, indices[idx])
        img = img.astype(float)/255
        opoints = add_corners_2(opoints)


        warped = morph(img, img, opoints.T, flattened_targ.T, simplices, 0, 1)

        warpeds.append(warped)

    canvas = np.zeros((224, 224, 3))
    for idx in range(BIG_K):
        canvas += warpeds[idx] * DISTRIBUTION[idx]
    
    canvas = (canvas / np.max(canvas) * 255).astype(np.uint8)
    
    skio.imsave(f"frames_tilt/{BREED}/frame_{frame_idx:03}.jpg", canvas)

    plt.figure(figsize=(4, 4))
    plt.gca().invert_yaxis()

    plt.scatter(*flattened_targ, s = 5)
    plt.triplot(flattened_targ[0], flattened_targ[1], simplices)
    plt.savefig(f"frames_tilt/{BREED}/wire_{frame_idx:03}.bmp")
    plt.close()

400it [14:59,  2.25s/it]


# for not_my_beagle

In [68]:
BREED = top_breeds[8]


alphas = [d['alpha'] for d in camera_mts[BREED]]
betas = [d['beta'] for d in camera_mts[BREED]]
gammas = [d['gamma'] for d in camera_mts[BREED]]
translates = [d['T'] for d in camera_mts[BREED]]
scales = [d['scale'] for d in camera_mts[BREED]]

betas_and_gammas = np.array([betas, gammas])

alpha, beta, gamma, scale, tx, ty = [-7.28233828e-03, -1.37024725e-01,  1.37066615e-02,  1.46087026e+00,
       -3.91845993e+01, -6.02513002e+01]

indices = get_top_k(betas_and_gammas, [beta, gamma])

three_d = add_corners_3(points_meanaligned_3d[BREED])

rotmx = reconstruct_tmx([alpha, beta,gamma, 1, 0, 0])[0]
ctr_pt = rotmx @ np.array([[112],[112],[0]])
three_d_spun = rotmx@three_d

new_width = three_d_spun[0, 15] - three_d_spun[0, 13]
new_height = three_d_spun[1, 14] - three_d_spun[1, 13]

# adjust the thing

scale_factor = 224/((new_width + new_height)/2)

three_d_spun, ctr_pt = three_d_spun * scale_factor, ctr_pt * scale_factor
three_d_spun = three_d_spun - ctr_pt + np.array([[112],[112],[0]])

flattened_targ = three_d_spun[:2]
flattened_targ = np.minimum(223, flattened_targ)
flattened_targ = np.maximum(0, flattened_targ)

warpeds = []
for idx in range(BIG_K):

    img, opoints = grab_pic(BREED, indices[idx])
    img = img.astype(float)/255
    opoints = add_corners_2(opoints)


    warped = morph(img, img, opoints.T, flattened_targ.T, simplices, 0, 1)

    warpeds.append(warped)

canvas = np.zeros((224, 224, 3))
for idx in range(BIG_K):
    canvas += warpeds[idx] * DISTRIBUTION[idx]

canvas = (canvas / np.max(canvas) * 255).astype(np.uint8)

skio.imsave(f"avg_my_angle.jpg", canvas)

plt.figure(figsize=(4, 4))
plt.gca().invert_yaxis()

plt.scatter(*flattened_targ, s = 5)
plt.triplot(flattened_targ[0], flattened_targ[1], simplices)
plt.savefig(f"frame_my_angle.jpg")
plt.close()

(flattened_targ)

array([[ 78.73665787, 134.63729151,  98.66707506,  70.81725375,
        111.60926102, 152.98989858,  70.99484282,  85.07996313,
        126.09116511, 142.17648083,  90.71008668, 105.64147476,
        100.73995678,   0.        ,   0.62899542, 222.36575011,
        223.        ],
       [100.594362  , 100.19219665, 145.2502091 ,  71.57652739,
         58.70429753,  70.97095535, 100.40835814, 103.18011896,
        102.94003263,  99.59805485, 145.14830304, 144.79483461,
        160.36233135,   0.        , 223.        ,   0.        ,
        223.        ]])

In [69]:
BREED = top_breeds[0]

alphas = [d['alpha'] for d in camera_mts[BREED]]
betas = [d['beta'] for d in camera_mts[BREED]]
gammas = [d['gamma'] for d in camera_mts[BREED]]
translates = [d['T'] for d in camera_mts[BREED]]
scales = [d['scale'] for d in camera_mts[BREED]]



alpha, beta, gamma, scale, tx, ty = [-7.28233828e-03, -1.37024725e-01,  1.37066615e-02,  1.46087026e+00,
       -3.91845993e+01, -6.02513002e+01]

indices = get_top_k(betas_and_gammas, [beta, gamma])

three_d = add_corners_3(points_meanaligned_3d[BREED])

rotmx = reconstruct_tmx([alpha, beta,gamma, 1, 0, 0])[0]
ctr_pt = rotmx @ np.array([[112],[112],[0]])
three_d_spun = rotmx@three_d

new_width = three_d_spun[0, 15] - three_d_spun[0, 13]
new_height = three_d_spun[1, 14] - three_d_spun[1, 13]

# adjust the thing

scale_factor = 224/((new_width + new_height)/2)

three_d_spun, ctr_pt = three_d_spun * scale_factor, ctr_pt * scale_factor
three_d_spun = three_d_spun - ctr_pt + np.array([[112],[112],[0]])

flattened_targ = three_d_spun[:2]
flattened_targ = np.minimum(223, flattened_targ)
flattened_targ = np.maximum(0, flattened_targ)

warpeds = []
for idx in range(BIG_K):

    img, opoints = grab_pic(BREED, indices[idx])
    img = img.astype(float)/255
    opoints = add_corners_2(opoints)


    warped = morph(img, img, opoints.T, flattened_targ.T, simplices, 0, 1)

    warpeds.append(warped)

canvas = np.zeros((224, 224, 3))
for idx in range(BIG_K):
    canvas += warpeds[idx] * DISTRIBUTION[idx]

canvas = (canvas / np.max(canvas) * 255).astype(np.uint8)

skio.imsave(f"bc_my_angle.jpg", canvas)

plt.figure(figsize=(4, 4))
plt.gca().invert_yaxis()

plt.scatter(*flattened_targ, s = 5)
plt.triplot(flattened_targ[0], flattened_targ[1], simplices)
plt.savefig(f"frame_bc_my_angle.jpg")
plt.close()

flattened_targ

array([[ 81.6017585 , 137.41219894, 105.54261924,  68.98820619,
        110.58177799, 153.43747501,  72.54974871,  87.96017914,
        128.90507542, 144.85226869,  95.92213173, 112.88097755,
        106.81419417,   0.        ,   0.62899542, 222.36575011,
        223.        ],
       [100.94662359, 100.52013288, 148.58361508,  52.93291958,
         45.10697333,  52.34763007,  99.68566094, 103.41281579,
        103.27140389,  99.07237396, 148.42166929, 148.29525272,
        164.89178216,   0.        , 223.        ,   0.        ,
        223.        ]])